# **Classification** des **séries temporelles** avec `sktime`

## Métadonnées

- **Expérience nº :** 7.2.2
- **Date :** 11/11/2023
- **Heure :** 12:28
- **Données :** `ready_Evaporation_Sunshine_Pressure9am_Pressure3pm_Cloud9am_Cloud3pm.csv`
- **Tri :** index chronologique
- **Découpage :** `TimeSeriesSplit`
- **Mise à l'échelle :** non
- **Rééquilibrage :** non
- **Conversion :** `numpyfy`
- **Algorithme :** TSF

## Sommaire

1. Initialisation
2. Découpage
3. Conversion
4. Modélisation
5. Évaluation

## 1. Initialisation

In [1]:
# Importation des bibliothèques et modules nécessaires au fonctionnement de ce notebook

import pandas as pd
import numpy as np

from sklearn.metrics import classification_report

In [2]:
# Importation du jeu de données et enregistrement dans le DataFrame `df`

df = pd.read_csv("../../../../data/processed/omar/ready_Evaporation_Sunshine_Pressure9am_Pressure3pm_Cloud9am_Cloud3pm.csv", index_col = 1).sort_index()
df = df.drop(columns = "Unnamed: 0")

In [3]:
# Inspection de la structure de `df`

df.head()

,MinTemp,MaxTemp,Rainfall,WindGustSpeed,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Temp9am,Temp3pm,...,year,month,day,LocationNum,WindGustDirNum,WindDir9amNum,WindDir3pmNum,Latitude,Longitude,CodeRegionNum
Date,,,,,,,,,,,,,,,,,,,,,
2008-12-01,15.2,32.6,0.0,59.000000,13.000000,22.000000,35.000000,23.000000,24.400000,31.300000,...,2008,12,1,8,2.748894,1.178097,3.141593,-33.751195,150.694171,0
2008-12-01,5.2,13.0,0.4,46.172309,15.912884,15.090231,77.114297,69.660632,6.905934,10.345942,...,2008,12,1,17,3.005216,3.013693,2.779950,-35.533333,148.783333,1
2008-12-02,15.0,31.7,0.0,28.000000,13.000000,15.000000,64.000000,38.000000,20.400000,30.100000,...,2008,12,2,8,0.392699,1.178097,1.570796,-33.751195,150.694171,0
2008-12-02,3.0,15.0,0.4,46.172309,26.000000,24.000000,66.000000,51.000000,7.100000,14.200000,...,2008,12,2,17,3.005216,3.141593,2.748894,-35.533333,148.783333,1
2008-12-03,15.9,33.4,0.0,52.000000,9.000000,30.000000,63.000000,16.000000,21.000000,31.400000,...,2008,12,3,8,2.356194,1.570796,2.356194,-33.751195,150.694171,0


In [4]:
# Inspection de la structure de `df`

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8668 entries, 2008-12-01 to 2017-06-25
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   MinTemp         8668 non-null   float64
 1   MaxTemp         8668 non-null   float64
 2   Rainfall        8668 non-null   float64
 3   WindGustSpeed   8668 non-null   float64
 4   WindSpeed9am    8668 non-null   float64
 5   WindSpeed3pm    8668 non-null   float64
 6   Humidity9am     8668 non-null   float64
 7   Humidity3pm     8668 non-null   float64
 8   Temp9am         8668 non-null   float64
 9   Temp3pm         8668 non-null   float64
 10  RainToday       8668 non-null   int64  
 11  RainTomorrow    8668 non-null   int64  
 12  year            8668 non-null   int64  
 13  month           8668 non-null   int64  
 14  day             8668 non-null   int64  
 15  LocationNum     8668 non-null   int64  
 16  WindGustDirNum  8668 non-null   float64
 17  WindDir9amNum   8668 no

## 2. Découpage

In [5]:
# Découpage de `df` sur l'axe des colonnes : séparation des variables explicatives (`data`) et cible (`target`)

data = df.drop(columns = "RainTomorrow")
target = df["RainTomorrow"]

In [6]:
# Découpage de `data` et de `target` sur l'axe des lignes : séparation des jeux d'entraînement (`*_train`) et de test (`*_test`) avec le splitter `TimeSeriesSplit`

from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits = 5) ## Nous fixons le paramètre `n_splits` à 5 afin d'avoir une répartition de 80 / 20 entre les jeux d'entraînement et de test, respectivement.

for train_index, test_index in tss.split(data):
    X_train, X_test = data.iloc[train_index, :], data.iloc[test_index,:]
    y_train, y_test = target.iloc[train_index], target.iloc[test_index]

## 3. Conversion

In [7]:
# Importation de la fonction artisanale `numpyfy`

from numpyfy import numpyfy

In [8]:
# Application de la fonction `numpyfy` aux 4 jeux de données issus du découpage effectué par le splitter `TimeSeriesSplit` afin de les convertir en arrays et ainsi les rendre compatibles avec `sktime`

X_train, X_test, y_train, y_test = numpyfy(X_train, X_test, y_train, y_test)

## 4. Modélisation

In [9]:
# Instanciation d'un modèle classificateur via un Pipeline

from sklearn.pipeline import Pipeline
from sktime.transformations.panel.compose import ColumnConcatenator
from sktime.classification.interval_based import TimeSeriesForestClassifier

steps = [
    ("concatenate", ColumnConcatenator()), ## Cette étape permet de transformer les jeux de données multivariées en univariées afin qu'ils puissent être traités par `TimeSeriesForestClassifier`.
    ("classify", TimeSeriesForestClassifier()),
]

clf_rf_ts = Pipeline(steps)

In [10]:
# Récupération des paramètres initiaux de la TSF

clf_rf_ts["classify"].get_params()

{'min_interval': 3, 'n_estimators': 200, 'n_jobs': 1, 'random_state': None}

In [11]:
# Entraînement du modèle

clf_rf_ts.fit(X_train, y_train)

Pipeline(steps=[('concatenate', ColumnConcatenator()),
                ('classify', TimeSeriesForestClassifier())])

In [12]:
# Récupération des paramètres ajustés de la TSF

clf_rf_ts["classify"].get_fitted_params()

{'classes': array([0, 1]),
 'intervals': [array([[14, 17],
         [12, 15],
         [ 7, 13],
         [13, 16]]),
  array([[ 6, 18],
         [17, 20],
         [ 7, 11],
         [ 9, 17]]),
  array([[13, 17],
         [ 2, 16],
         [ 8, 15],
         [14, 17]]),
  array([[ 3, 11],
         [ 5, 17],
         [11, 19],
         [12, 15]]),
  array([[ 4, 19],
         [ 0,  3],
         [ 6, 15],
         [15, 18]]),
  array([[16, 19],
         [ 4, 15],
         [ 8, 18],
         [ 5, 16]]),
  array([[12, 18],
         [10, 15],
         [ 6, 14],
         [11, 16]]),
  array([[ 0,  3],
         [ 3, 18],
         [ 9, 16],
         [ 5,  9]]),
  array([[17, 20],
         [ 8, 11],
         [ 6, 16],
         [ 5,  8]]),
  array([[ 3, 16],
         [12, 15],
         [ 6,  9],
         [12, 15]]),
  array([[ 7, 12],
         [ 8, 13],
         [12, 16],
         [ 9, 17]]),
  array([[12, 19],
         [ 9, 16],
         [ 7, 17],
         [12, 18]]),
  array([[17, 20],
     

In [13]:
# Réalisation des prédictions

y_pred = clf_rf_ts.predict(X_test)

## 5. Évaluation

In [14]:
# Élaboration de la matrice de confusion

pd.crosstab(y_test, y_pred, rownames = ["Classe réelle"], colnames = ["Classe prédite"])

Classe prédite,0,1
Classe réelle,,
0,1115,33
1,180,116


In [15]:
# Élaboration du rapport de classification

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1148
           1       0.78      0.39      0.52       296

    accuracy                           0.85      1444
   macro avg       0.82      0.68      0.72      1444
weighted avg       0.84      0.85      0.83      1444

